# import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


# Importing libraries

In [2]:
world_cup = pd.read_csv("datasets/World Cup 2018 Dataset.csv")
results = pd.read_csv("datasets/results.csv")

In [3]:
winner = []
for i in range (len(results['home_team'])):
    if results['home_score'][i] > results['away_score'][i]:
        winner.append(results['home_team'][i])
    elif results['home_score'][i] < results['away_score'][i]:
        winner.append(results['away_team'][i])
    else:
        winner.append("Draw")
results['winning_team'] = winner

#adding goal differe column 

results['goal_difference']= np.absolute(results['home_score']-results['away_score'])
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team,goal_difference
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,Draw,0
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England,2
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,Scotland,1
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,Draw,0
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,Scotland,3


In [4]:
# work with subset of data

df=results[(results['home_team']=='Nigeria') | (results['away_team'] =='Nigeria')]
nigeria = df.iloc[:]
nigeria.head()


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team,goal_difference
2977,1949-10-08,Sierra Leone,Nigeria,0,2,Friendly,Freetown,Sierra Leone,False,Nigeria,2
3050,1950-05-28,Ghana,Nigeria,1,0,Friendly,Accra,Gold Coast,False,Ghana,1
3219,1951-10-20,Nigeria,Ghana,5,0,Friendly,Lagos,Nigeria,False,Nigeria,5
3492,1953-10-11,Ghana,Nigeria,1,0,Friendly,Accra,Gold Coast,False,Ghana,1
3654,1954-10-30,Nigeria,Ghana,3,0,Friendly,Lagos,Nigeria,False,Nigeria,3


In [5]:
year =[]
for row in nigeria['date']:
    year.append(int(row[:4]))
    
nigeria['match_year'] = year
nigeria_1930 = nigeria[nigeria.match_year >= 1930]
nigeria_1930.count()


date               548
home_team          548
away_team          548
home_score         548
away_score         548
tournament         548
city               548
country            548
neutral            548
winning_team       548
goal_difference    548
match_year         548
dtype: int64

In [6]:
worldcup_teams =['Australia', 'Iran','Japan','Korea Republic',
                 'Saudi Arabia', 'Egypt','Morocco',
                 'Nigeria', 'Senegal','Tunisia',
                 'Costa Rica','Mexico', 'Panama','Argentina'
                 ,'Brazil', 'Colombia' 'Peru',
                 'Uruguay', 'Belgium', 'croatia', 'Denmark',
                 'England','France', 'Iceland',
                 'Poland', 'Portugal','Russia',
                 'Serbia','Spain', 'Germany', 'sweden',
                 'switzerland']

In [7]:
len(worldcup_teams)

31

In [8]:
df_teams_home = results[results['home_team'].isin(worldcup_teams)]
df_teams_away = results[results['away_team'].isin(worldcup_teams)]
df_teams =pd.concat ((df_teams_home, df_teams_away)) 
df_teams.drop_duplicates() 
df_teams.count()

date               18058
home_team          18058
away_team          18058
home_score         18058
away_score         18058
tournament         18058
city               18058
country            18058
neutral            18058
winning_team       18058
goal_difference    18058
dtype: int64

In [9]:
#create an year column to drop games before 1930 
year=[]
for row in df_teams ['date']: 
    year.append (int (row[:4]))
df_teams['match_year']=year
df_teams_1930 = df_teams[df_teams.match_year>=1930]
df_teams_1930.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team,goal_difference,match_year
1230,1930-01-01,Spain,Czechoslovakia,1,0,Friendly,Barcelona,Spain,False,Spain,1,1930
1231,1930-01-12,Portugal,Czechoslovakia,1,0,Friendly,Lisbon,Portugal,False,Portugal,1,1930
1237,1930-02-23,Portugal,France,2,0,Friendly,Porto,Portugal,False,Portugal,2,1930
1238,1930-03-02,Germany,Italy,0,2,Friendly,Frankfurt am Main,Germany,False,Italy,2,1930
1240,1930-03-23,France,Switzerland,3,3,Friendly,Colombes,France,False,Draw,0,1930


In [10]:
#drop col that they dont influence matches
df_teams_1930 = df_teams.drop(['date','home_score','away_score',
                              'tournament','city','country'])
df_teams_1930.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team,goal_difference,match_year
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England,2,1873
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,Draw,0,1875
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,False,Scotland,2,1877
10,1879-01-18,England,Wales,2,1,Friendly,London,England,False,England,1,1879
11,1879-04-05,England,Scotland,5,4,Friendly,London,England,False,England,1,1879


In [11]:
#build a model

df_teams_1930 = df_teams_1930.reset_index(drop=True)
df_teams_1930.loc[df_teams_1930.winning_team == df_teams_1930.home_team,'winning_team']=2
df_teams_1930.loc[df_teams_1930.winning_team == 'Draw','winning_team']=1
df_teams_1930.loc[df_teams_1930.winning_team ==df_teams_1930.away_team,'winning_team']=0
df_teams_1930.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team,goal_difference,match_year
0,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,2,2,1873
1,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,1,0,1875
2,1877-03-03,England,Scotland,1,3,Friendly,London,England,False,0,2,1877
3,1879-01-18,England,Wales,2,1,Friendly,London,England,False,2,1,1879
4,1879-04-05,England,Scotland,5,4,Friendly,London,England,False,2,1,1879


In [19]:
final =pd.get_dummies(df_teams_1930,prefix=['home_team','away_team'],columns=['home_team','away_team'])
X=final.drop(['winning_team'],axis=1)
y=final['winning_team']
y=y.astype('int')

x_train,x_test,y_train,y_test =train_test_split(X,y,random_state=42)

In [20]:
logreg= LogisticRegression()
logreg.fit(x_train,y_train)
# print(logreg.score(x_test,y_test))

ValueError: could not convert string to float: 'Liechtenstein'